# T1001.002 Steganography

-----------------------------------------------------------------------

## Technique Description

Adversaries may use steganographic techniques to hide command and control traffic to make detection efforts more difficult. Steganographic techniques can be used to hide data in digital messages that are transferred between systems. This hidden information can be used for command and control of compromised systems. In some cases, the passing of files embedded using steganography, such as image or document files, can be used for command and control. 

## Technique Detection

Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used.(Citation: University of Birmingham C2)

-----------------------------------------------------------------------

### Tactics:

  *   Command-And-Control

### Platforms:

  * Linux

  * macOS

  * Windows

### Data Sources:

  * **Network Traffic:** Network Traffic Content

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| APT29 | [APT29](https://attack.mitre.org/groups/G0016) has used steganography to hide C2 communications in images.(Citation: ESET Dukes October 2019)| 
| Axiom | [Axiom](https://attack.mitre.org/groups/G0001) has used steganography to hide its C2 communications.(Citation: Novetta-Axiom)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1001/002)

  * [University Of Birmingham C2](https://arxiv.org/ftp/arxiv/papers/1408/1408.1136.pdf), Gardiner, J.,  Cova, M., Nagaraja, S. (2014, February). Command & Control Understanding, Denying and Detecting. Retrieved April 20, 2016.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 13 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres, CTR Emily Porras

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Adversaries may use steganographic techniques to hide command and control traffic to make detection efforts more difficult. Steganographic techniques can be used to hide data in digital messages that are transferred between systems. This hidden information can be used for command and control of compromised systems. In some cases, the passing of files embedded using steganography, such as image or document files, can be used for command and control.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Adversary  | X | X |

## Detection Blindspots

- Incorrect sensor placement may not allow for this TTP to be identified if required traffic is not captured.
- Encryption will make analysis of payload difficult and connection metadata should be utilized.
- Identification of stego tool used will aid in reversing the stego process
- Hex analysis and carving may be required to pull the data out of the stego file

## Analytical References

  * [Operation Ghost Dukes](https://www.welivesecurity.com/wp-content/uploads/2019/10/ESET_Operation_Ghost_Dukes.pdf)
  * [ATT&CK with Corelight](https://www.corelight.com/mitre-attack/c2/t1001-data-obfuscation)

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

#### Analytic 2

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Analyze network data for uncommon data flows (e.g., a client sending significantly more data than it receives from a server). Processes utilizing the network that do not normally have network communication or have never been seen before are suspicious. Analyze packet contents to detect communications that do not follow the expected protocol behavior for the port that is being used.
- Data obfuscation, including steganography, often happens within transferred files. Examples include malware like the Duqu, HAMMERTOSS, and ZeroT malware families, which insert or append Command and Control (C2) information into image files.
- Corelight can extract files from network streams, with file extraction enabled, allowing defenders to scrutinize transferred files for anomalies like steganographic data. Additionally, Corelight summarizes and catalogs protocols regardless of the port. If the adversary has modified a standard protocol then Zeek's Dynamic Protocol Detection (DPD) gives defenders a starting point to identify protocol violations.

#### Analytic 1
  * **Information:** Analyze HTTP traffic with high entropy.

  * **Source:** PCAP

  * **Tool:** Arkime

  * **Notes:** This TTP is extermely difficult to detect and reverse. Often times the adversary will utilize multiple layers of obfuscation to hide their communications

  * **Query Arkime:** 
    -`protocols == http && http.uri == EXISTS! && http.host == EXISST! && http.method == [GET, POST] && http.statuscode == 200`
    -`entropy.http == [6,7,8]`
      * look for high entropy successful http connections
      * you can change the EXISTS! functions to host/uri of interest if known

#### Analytic 2

  * **Information:** Analyze HTTP traffic with file names and libfile content type

  * **Source:** PCAP

  * **Tool:** Arkime

  * **Notes:** This TTP is extermely difficult to detect and reverse. Often times the adversary will utilize multiple layers of obfuscation to hide their communications

  * **Query Arkime:** 
    -`protocols == http && http.method == POST && http.statuscode == 200 && file == EXISTS!`
    -`entropy.http == [6,7,8]`
      * look for high entropy successful http connections
      * where the filename exists and libfile content type is parsed
      * may be able to use cyberchef to identify more information about the file
      * IOCs of known C2 domains or IP can help corroborate your findings

#### Analytic 3

  * **Information:** Analyze Zeek log files.

  * **Source:** PCAP

  * **Tool:** Zeek

  * **Notes:** This TTP is extermely difficult to detect and reverse. Often times the adversary will utilize multiple layers of obfuscation to hide their communications

  * **Zeek Analysis:** 
    - Analyze dpd.log
        * uuid - connection unique ID
        * analyzer - the analyzer that generated the violation
        * failure_reason - the textual reason for the analysis failure
    - Analyze conn.log
        * uuid - unique identifier of connection
        * service - application protocol ID sent over connection
        * resp_btyes - number of payload bytes responder sent
        * history - connection stat history
    - Analyze files.log
        * FUID - identifier associated with a single file
        * conn_uids - connection UID(s) over which file transferred
        * mime_type - file type, as determined by Bro's signatures
        * md5 - digest of the file contents
        * sha1 - digest of file contents
        * sha256 - digest of file contents
    - Analyze http.log
        * ts - date time stamp
        * uid & id - underlying connection info
        * method - verb used in HTTP request (GET, POST)
        * host - value of HOST header
        * uri - uri used in request
        * referrer - value of "referrer" header
        * user_agent - vlaue of version portion of request
    - Analyze smtp.log
        * uid & id - underlying connection info
        * mailfrom - email addresses found in From header
        * msg_id - contents of MsgID header
        * x_orignating_ip - contents of X_Originating-IP header
        * is_webmail - if the message was sent via webmail (boolean)